In [1]:
### source: https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173
### consider calculating the precision and recall of each exam + between all validation exams 
### also the precision-recall curve
### also consider the recall-IoU curve

In [2]:
import numpy as np
from tqdm import tqdm
import torch
from generate_transforms import generate_detection_inference_transform
from monai.apps.detection.networks.retinanet_detector import RetinaNetDetector
from monai.apps.detection.utils.anchor_utils import AnchorGeneratorWithAnchorShape
from monai.data import DataLoader, Dataset, load_decathlon_datalist
from monai.data.utils import no_collation
from monai.transforms import ScaleIntensityRanged

In [3]:
gt_box_mode = "cccwhd"
data_list_file_path = "/data/output/mixed_data/mixed_test.json"
data_base_dir = "/data/mixed/"

returned_layers = [1,2]
base_anchor_shapes = [[6,8,4],[8,6,5],[10,10,6]]
patch_size = [192,192,80]
amp = True

model_path = "/mlflow/5/93273c7eb040429a9470b98ebd6ffb12/artifacts/best_model/data/model.pth"
score_thresh = 0.02
nms_thresh = 0.22

In [4]:
intensity_transform = ScaleIntensityRanged(
    keys=["image"],
    a_min=-1024,
    a_max=300.0,
    b_min=0.0,
    b_max=1.0,
    clip=True,
)
inference_transforms, post_transforms = generate_detection_inference_transform(
    "image",
    "pred_box",
    "pred_label",
    "pred_score",
    gt_box_mode,
    intensity_transform,
    affine_lps_to_ras=True,
    amp=amp,
)
inference_data = load_decathlon_datalist(
    data_list_file_path,
    is_segmentation=True,
    data_list_key="test",
    base_dir=data_base_dir,
)
inference_ds = Dataset(
    data=inference_data,
    transform=inference_transforms,
)
inference_loader = DataLoader(
    inference_ds,
    batch_size=1,
    pin_memory=torch.cuda.is_available(),
    collate_fn=no_collation,
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
anchor_generator = AnchorGeneratorWithAnchorShape(
    feature_map_scales=[2**l for l in range(len(returned_layers) + 1)],
    base_anchor_shapes=base_anchor_shapes,
)

In [7]:
net = torch.jit.load(model_path).to(device)
print(f"Load model from {model_path}")

Load model from /mlflow/5/93273c7eb040429a9470b98ebd6ffb12/artifacts/best_model/data/model.pth


In [8]:
detector = RetinaNetDetector(
    network=net, anchor_generator=anchor_generator, debug=False
)

In [9]:
detector.set_box_selector_parameters(
    score_thresh=score_thresh,
    topk_candidates_per_level=1000,
    nms_thresh=nms_thresh,
    detections_per_img=100,
)
detector.set_sliding_window_inferer(
    roi_size=patch_size,
    overlap=0.25,
    sw_batch_size=1,
    mode="gaussian",
    device="cpu",
)

In [10]:
%%time
results_dict = {"test": []}
detector.eval()

with torch.no_grad():
    for inference_data in tqdm(inference_loader):
        inference_img_filenames = [
            inference_data_i["image_meta_dict"]["filename_or_obj"] for inference_data_i in inference_data
        ]
        print(inference_img_filenames)
        use_inferer = not all(
            [inference_data_i["image"][0, ...].numel() < np.prod(patch_size) for inference_data_i in inference_data]
        )
        inference_inputs = [inference_data_i["image"].to(device) for inference_data_i in inference_data]

        if amp:
            with torch.cuda.amp.autocast():
                inference_outputs = detector(inference_inputs, use_inferer=use_inferer)
        else:
            inference_outputs = detector(inference_inputs, use_inferer=use_inferer)
        del inference_inputs

        # update inference_data for post transform
        for i in range(len(inference_outputs)):
            inference_data_i, inference_pred_i = (
                inference_data[i],
                inference_outputs[i],
            )
            inference_data_i["pred_box"] = inference_pred_i[detector.target_box_key].to(torch.float32)
            inference_data_i["pred_label"] = inference_pred_i[detector.target_label_key]
            inference_data_i["pred_score"] = inference_pred_i[detector.pred_score_key].to(torch.float32)
            inference_data[i] = post_transforms(inference_data_i)

        for inference_img_filename, inference_pred_i in zip(inference_img_filenames, inference_data):
            result = {
                "label": inference_pred_i["pred_label"].cpu().detach().numpy().tolist(),
                "box": inference_pred_i["pred_box"].cpu().detach().numpy().tolist(),
                "score": inference_pred_i["pred_score"].cpu().detach().numpy().tolist(),
            }
            result.update({"image": inference_img_filename})
            results_dict["test"].append(result)

['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.252697338970999211181671881792/1.3.6.1.4.1.14519.5.2.1.6279.6001.252697338970999211181671881792.nii.gz']
['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.797637294244261543517154417124/1.3.6.1.4.1.14519.5.2.1.6279.6001.797637294244261543517154417124.nii.gz']
['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.150684298696437181894923266019/1.3.6.1.4.1.14519.5.2.1.6279.6001.150684298696437181894923266019.nii.gz']
['/data/mixed/PL155113087338785/PL155113087338785.nii.gz']
['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.413896555982844732694353377538/1.3.6.1.4.1.14519.5.2.1.6279.6001.413896555982844732694353377538.nii.gz']
['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.227962600322799211676960828223/1.3.6.1.4.1.14519.5.2.1.6279.6001.227962600322799211676960828223.nii.gz']
['/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.188619674701053082195613114069/1.3.6.1.4.1.14519.5.2.1.6279.6001.188619674701053082195613114069.nii.gz']
['/data/mixed/lung_048/

In [12]:
# import json

# with open("mixed_test.json", "w") as write_file:
#     json.dump(results_dict, write_file, indent=4)